In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

../data/sample_submission.csv
../data/test.csv
../data/train.csv


In [2]:
import torch
from torch import nn
from torch.utils.data import Dataset, random_split, DataLoader
from torchvision.transforms import ToTensor, Lambda, Normalize, Compose
from torch.optim.lr_scheduler import ReduceLROnPlateau

import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from tqdm import trange

In [3]:
class CustomDataset(Dataset):
    def __init__(self, input_file, device=torch.device('cpu'), image_size=(28,28), num_labels=10, train=True):
        self.df = pd.read_csv(input_file)
        self.image_size = image_size
        self.train = train
        self.device = device
        # print(input_file, image_size, num_labels, train)
        self.transform = Compose([
            ToTensor(),
            Normalize((0.5,), (0.5,))
        ])
        self.target_transform_bkp = Lambda(
            lambda y: torch.zeros(num_labels, dtype=torch.float).scatter_(0,torch.tensor(y), value=1)
        )
        self.target_transform = Lambda(
            lambda y: torch.tensor(y, dtype=torch.long)
        )
        

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        input_image, input_label = None, None
        if(self.train):
            input_image = self.df.iloc[idx, 1:].to_numpy().reshape(self.image_size)
            input_label = self.df.iloc[idx, 0]
        else:
            input_image = self.df.iloc[idx, :].to_numpy().reshape(self.image_size)

        input_image = input_image.astype(np.float32)
        input_image = self.transform(input_image).to(torch.float).to(device)
        if(input_label is not None):input_label = self.target_transform(input_label).to(device)

        # print(input_image.dtype)

        return {
            "image": input_image, "label":input_label
        }


In [4]:
def evaluate(model, dataloader, loss_fn, batch_size, evaluation_prefix=''):
    model.eval()
    
    loss, correct = 0, 0
    dataset_size = len(dataloader.dataset)
    num_batches = len(dataloader)
    
    with torch.no_grad():
        for input_batch in tqdm(dataloader, dynamic_ncols=True, desc=f"{evaluation_prefix} Evaluate", leave=False): #
            images, labels = input_batch['image'], input_batch['label']
    
            logits = model(images)
            # probabilities = nn.functional.softmax(logits, dim=1)
            
            loss += loss_fn(logits, labels)
            correct += ( logits.argmax(1) == labels ).type(torch.float).sum().item()

        loss /= num_batches
        correct /= dataset_size

        print(f"{evaluation_prefix} Evaluation : loss={loss.item():>8f} Accuracy={correct*100:>.1f} %")

    model.train()

    return correct*100, loss

In [29]:
!mkdir ../modelWeights/ -p

In [28]:
def train(
    model,
    train_dataloader, val_dataloader,
    epochs=1,
    learning_rate=1e-3,
    epoch_offset=0
    
):
    dataset_size = len(train_dataloader.dataset)
    loss = 0 # binary cross entropy
    loss_fn = nn.CrossEntropyLoss()
    optimizer = None # Adam, RMSProp
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=3)
    
    model.train()
    for eidx in range(1,epochs+1):
        # print(f"Epoch: {eidx}")
        pbar = tqdm(train_dataloader, desc=f"Epoch: {eidx+epoch_offset}", dynamic_ncols=True)
        for batch, input_batch in enumerate(pbar):
            images, labels = input_batch['image'], input_batch['label']
    
            logits = model(images)
            # probabilities = nn.functional.softmax(logits, dim=1)

            loss = loss_fn(logits, labels)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            if(batch % 10 == 0):
                loss_value = loss.item()
                pbar.set_postfix_str(f"loss: {loss_value:>7f}")
                # current = batch*batch_size+len(images)
                # print(f"epoch : {eidx}:{current*100/dataset_size:02.2f}, train loss : {loss_value}, ")
                # print(f"loss: {loss_value:>7f}  [{current:>5d}/{dataset_size:>5d}]")
                


        train_acc, train_loss = evaluate(model, train_dataloader, loss_fn, batch_size, evaluation_prefix="Training data")
        val_acc, val_loss = evaluate(model, val_dataloader, loss_fn, batch_size, evaluation_prefix="Validation data")

        old_lr = optimizer.param_groups[0]['lr']
        scheduler.step(val_loss)
        new_lr = optimizer.param_groups[0]['lr']
        
        print(f"lr : {old_lr} -> {new_lr}")
        
        model_output_path = f"../modelWeights/SimpleANN_epoch{eidx+epoch_offset:02d}_Acc{val_acc:>.1f}.pth"
        # model_output_path = f"./modelWeights/SimpleANN_epoch{eidx+epoch_offset:02d}.pth"
        print(f"saving : {model_output_path}")
        torch.save(model.state_dict(), model_output_path)

In [7]:
def predict(model, device, dataset):
    model.eval()
    outputs = pd.DataFrame(columns=["ImageId","Label"])
    with torch.no_grad():
        for idx in tqdm(range(len(dataset)), dynamic_ncols=True, desc="Predict"):
            image = dataset[idx]['image']
            logits = model(image)
            prob = torch.softmax(logits, dim=1)
            pred_class = prob.argmax(1).cpu().item()
            outputs.loc[len(outputs)] = [(idx+1), pred_class]

    return outputs


In [8]:
def showData(dataset):
    cols, rows = 5, 5

    fig = plt.figure(figsize=(10,10))
    
    for idx in range(1,cols*rows+1):
        sample_idx = torch.randint(len(dataset), size=(1,)).item()
        sample = dataset[sample_idx]
        image, label = sample['image'], sample['label']
        sub_title = label.item() if(label is not None) else idx 
        fig.add_subplot(rows, cols, idx)
        plt.title(sub_title)
        plt.axis('off')
        plt.imshow(image.squeeze().cpu(), cmap='gray')


In [23]:
class SimpleANN(nn.Module):
    def __init__(self, input_image_size=(28,28)):
        super().__init__()
        self.flatten = nn.Flatten()
        self.LinearRelu_1 = nn.Sequential(
            nn.Linear(input_image_size[0]*input_image_size[1], 512),
            nn.Dropout(0.2),
            nn.ReLU()
        )
        self.LinearRelu_2 = nn.Sequential(
            nn.Linear(512, 512),
            nn.Dropout(0.2),
            nn.ReLU()
        )
        self.Linear_out = nn.Linear(512, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.LinearRelu_1(x)
        x = self.LinearRelu_2(x)
        logits = self.Linear_out(x)
        
        return logits


In [10]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device


device(type='cuda')

In [11]:
# !ls /kaggle/input/digitrecognizer_simpleann_1/pytorch/default/1/

In [12]:
batch_size=64
loss_fn = nn.CrossEntropyLoss()

In [13]:
dataset = CustomDataset(input_file='../data/train.csv', device=device, train=True)

In [14]:
val_perc = 0.2
train_perc = 1 - val_perc

generator = torch.Generator().manual_seed(42)
train_dataset, val_dataset = random_split(dataset, [train_perc, val_perc], generator)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)

In [15]:
input_batch = dataset[0]

In [25]:
model = SimpleANN()
model.to(device)
# model.load_state_dict(torch.load("/kaggle/input/digitrecognizer_simpleann_1/pytorch/default/1/SimpleANN_epoch10.pth", weights_only=True))
evaluate(model, val_dataloader, loss_fn, batch_size, "Validation data");

Validation data Evaluate:   0%|                                  | 0/132 [00:00<?, ?it/s]

Validation data Evaluation : loss=24.976805 Accuracy=11.3 %


In [26]:
# !rm -rf ./modelWeights/*.pth

In [27]:
train(
    model,
    train_dataloader, val_dataloader,
    epochs=50, learning_rate=1e-3, epoch_offset=0,
)

Epoch: 1:   0%|                                                  | 0/525 [00:00<?, ?it/s]

Training data Evaluate:   0%|                                    | 0/525 [00:00<?, ?it/s]

Training data Evaluation : loss=0.194930 Accuracy=94.5 %


Validation data Evaluate:   0%|                                  | 0/132 [00:00<?, ?it/s]

Validation data Evaluation : loss=0.279293 Accuracy=92.9 %
lr : 0.001 -> 0.001
saving : ./modelWeights/SimpleANN_epoch01_Acc92.9.pth


...

Epoch: 50:   0%|                                                 | 0/525 [00:00<?, ?it/s]

Training data Evaluate:   0%|                                    | 0/525 [00:00<?, ?it/s]

Training data Evaluation : loss=0.002639 Accuracy=100.0 %


Validation data Evaluate:   0%|                                  | 0/132 [00:00<?, ?it/s]

Validation data Evaluation : loss=0.127312 Accuracy=97.2 %
lr : 1.5625e-05 -> 1.5625e-05
saving : ./modelWeights/SimpleANN_epoch50_Acc97.2.pth


In [ ]:
outputs = predict(model, device, test_dataset)

In [ ]:
outputs.head()

In [ ]:
outputs.to_csv("./output_SimpleANN_epoch50.csv",index=False)

!head -n6 ./output_SimpleANN_epoch25_Acc97.5.csv